In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import random
import time
import pandas as pd
from datetime import datetime
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl.utils import escape
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE


In [3]:
import os

In [4]:
from selenium.webdriver.common.action_chains import ActionChains

In [5]:
chrome_driver_path = "D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\nhatk\AppData\Local\Temp\ipykernel_12556\211932188.py:1: SyntaxWarning: invalid escape sequence '\c'
  chrome_driver_path = "D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"


In [ ]:
options = Options()
options.headless = True  # không hiển thị trình duyệt
options.add_argument("--disable-gpu")
options.add_argument("--blink-settings=imagesEnabled=false")  # tắt hình ảnh
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/115.0 Safari/537.36")

In [49]:
# Khởi tạo driver
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)

In [10]:
# Mở trang Cafef
driver.get("https://cafef.vn")

In [11]:
def clean_text(text):
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub("", text)
    return text

In [12]:
def convert_time_format(raw_time):
    try:
        # Bỏ phần ngày trong tuần (VD: "Thứ ba,") và phần (GMT+7)
        clean_time = re.sub(r'^.*?,\s*', '', raw_time)
        clean_time = re.sub(r'\s*\(.*?\)', '', clean_time)      # Bỏ "(GMT+7)"
        clean_time = clean_time.strip()
        print(clean_time)

        # Chuyển đổi thành datetime
        dt = datetime.strptime(clean_time, "%d/%m/%Y, %H:%M")

        return dt.strftime("%d/%m/%Y %H:%M")
    except Exception as e:
        print(f"Lỗi chuyển đổi thời gian: {e}")
        return raw_time

In [13]:
def crawl_news_vnexpress(ticker='VIC', start_date='01/01/2024 00:00', end_date='01/05/2025 23:59', max_pages=5, delay=2):
    """
    Crawl tin tức từ VnExpress theo từ khóa (ticker) sử dụng Selenium và trả về DataFrame.

    Args:
        ticker (str): Từ khóa tìm kiếm, ví dụ 'VIC'.
        max_pages (int): Số trang kết quả tìm kiếm muốn thu thập.
        delay (int): Thời gian chờ giữa các trang (giây).

    Returns:
        pd.DataFrame: DataFrame chứa các cột: title, ticker, time, url, Content.
    """


    base_url = f"https://timkiem.vnexpress.net/?search_q={ticker}&cate_code=kinhdoanh&media_type=all&latest=on&fromdate=&todate=&date_format=all"
    start_dt = datetime.strptime(start_date, "%d/%m/%Y %H:%M")
    end_dt = datetime.strptime(end_date, "%d/%m/%Y %H:%M")

    results = []

    for page in range(1, max_pages + 1):
        driver.get(f"{base_url}&page={page}")
        print(f"Đã vào xử lý trang {page}...")
        time.sleep(delay)

        articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news[data-url]")

        if not articles:
            print(f"Không có bài viết nào trên trang {page}.")
            break

        for article in articles:
            try:
                print(article.text.strip())
                title_tag = article.find_elements(By.CSS_SELECTOR, "h3.title-news a")[0]
                title = title_tag.text.strip()
                url = title_tag.get_attribute("href")
                print(f"Đang xử lý bài viết: {title}")
                print(f"URL: {url}")



                # Truy cập từng bài viết để lấy nội dung
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[1])
                driver.get(url)
                print(f"Truy cập bài viết: {url}")
                time.sleep(delay)

                # Lấy thời gian đăng
                try:
                    time_tag = driver.find_element(By.CSS_SELECTOR, "span.date")
                    time_text = time_tag.text.strip()
                    time_text = convert_time_format(time_text)
                except Exception as e:
                    time_text = ""
                    print(f"Lỗi lấy thời gian: {e}")
                print(f"Thời gian đăng: {time_text}")

                # Kiểm tra điều kiện thời gian
                pub_time_date = datetime.strptime(time_text, "%d/%m/%Y %H:%M")
                if pub_time_date > end_dt:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"Bài viết quá mới không nằm trong khoản thời gian cần crawl: {time_text}")
                    continue
                if pub_time_date < start_dt:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"Bài viết quá cũ không nằm trong khoản thời gian cần crawl: {time_text}")
                    return pd.DataFrame(results)



                # Lấy nội dung bài viết (cả mô tả nếu có)
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, "article.fck_detail p")
                    description = driver.find_elements(By.CSS_SELECTOR, "p.description")
                    content_parts = []

                    if description:
                        content_parts.extend([desc.text.strip() for desc in description if desc.text.strip()])
                    content_parts.extend([p.text.strip() for p in paragraphs if p.text.strip()])

                    content = "\n".join(content_parts)

                except:
                    content = ""

                print(f"Nội dung bài viết: {content}")


                # Đóng tab bài viết và quay lại
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print("Đã đóng tab bài viết")

                results.append({
                    'ticker': ticker,
                    'time': time_text,
                    'tittle': title,
                    'url': url,
                    'Content': clean_text(content)
                })
                print("Đã thêm vào kết quả")

            except Exception as e:
                print(f"Lỗi khi xử lý bài viết: {e}")
                continue

    # driver.quit()
    df = pd.DataFrame(results)
    return df

In [14]:
def convert_time_format_vietnamnet(raw_time):
    """
    Chuyển định dạng 'Thứ Ba, 13/05/2025 - 05:35' => '13/05/2025 05:35'
    """
    try:
        # Loại bỏ "Thứ ...," và tách theo dấu "-"
        clean_time = re.sub(r'^.*?,\s*', '', raw_time)  # "Thứ Ba, " -> ""
        date_part, time_part = clean_time.split(" - ")
        dt = datetime.strptime(f"{date_part.strip()} {time_part.strip()}", "%d/%m/%Y %H:%M")
        return dt.strftime("%d/%m/%Y %H:%M")
    except Exception as e:
        print(f"Lỗi chuyển đổi thời gian: {e}")
        return raw_time

In [ ]:
def crawl_news_vietnamnet(ticker='VIC', start_date='01/01/2024 00:00', end_date='01/05/2025 23:59', max_pages=3, delay=2):
    start_dt = datetime.strptime(start_date, "%d/%m/%Y %H:%M")
    end_dt = datetime.strptime(end_date, "%d/%m/%Y %H:%M")
    results = []

    for page in range(0, max_pages):
        print(f"Đang xử lý trang {page+1}...")
        search_url = f"https://vietnamnet.vn/tim-kiem-p{page}?bydaterang=all&cate=000003&newstype=all&od=2&q={ticker}"
        driver.get(search_url)
        time.sleep(delay)

        articles = driver.find_elements(By.CSS_SELECTOR, "div.horizontalPost")

        if not articles:
            print(f"Không có bài viết nào trên trang {page}.")
            break

        for article in articles:
            try:
                a_tag = article.find_elements(By.CSS_SELECTOR, "div.horizontalPost__avt a")[0]
                title = a_tag.get_attribute("title")
                url = a_tag.get_attribute("href")
                print(f"Đang xử lý: {title}")
                print(f"URL: {url}")

                # Mở bài viết để lấy nội dung và thời gian
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[1])
                driver.get(url)
                time.sleep(delay)

                # Thời gian
                try:
                    time_element = driver.find_element(By.CSS_SELECTOR, "div.bread-crumb-detail__time")
                    time_text = convert_time_format_vietnamnet(time_element.text.strip())
                except:
                    time_text = ""
                print(f"Thời gian đăng: {time_text}")

                # Kiểm tra điều kiện thời gian
                pub_time_date = datetime.strptime(time_text, "%d/%m/%Y %H:%M")
                if pub_time_date > end_dt:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"Bài viết quá mới không nằm trong khoản thời gian cần crawl: {time_text}")
                    continue
                if pub_time_date < start_dt:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"Bài viết quá cũ không nằm trong khoản thời gian cần crawl: {time_text}")
                    return pd.DataFrame(results)

                # Nội dung
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.maincontent p")
                    description = driver.find_elements(By.CSS_SELECTOR, "h2.content-detail-sapo")
                    content_parts = []

                    if description:
                        content_parts.extend([desc.text.strip() for desc in description if desc.text.strip()])
                    content_parts.extend([p.text.strip() for p in paragraphs if p.text.strip()])

                    content = "\n".join(content_parts)
                except:
                    content = ""
                print(f"Nội dung bài viết: {content}")
                # Đóng tab
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print("Đã đóng tab bài viết")


                results.append({
                    'ticker': ticker,
                    'time': time_text,
                    'tittle': title,
                    'url': url,
                    'Content': clean_text(content)
                })
                print("Đã thêm vào kết quả")

            except Exception as e:
                print(f"Lỗi xử lý bài viết: {e}")
                continue

    # driver.quit()
    return pd.DataFrame(results)

In [23]:
def init_driver():
    options = Options()
    options.headless = True  # không hiển thị trình duyệt
    options.add_argument("--disable-gpu")
    options.add_argument("--blink-settings=imagesEnabled=false")  # tắt hình ảnh
    options.add_argument("--disable-extensions")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/115.0 Safari/537.36")
    # Khởi tạo driver
    driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)
    return driver

In [27]:
driver = init_driver()

In [30]:
file_path = 'D:/thacsi/TAILIEULUANVAN/code/PredictStock_TA_FA_SA/SA/v2/data/news_vnexpress_vietnamnet_tuoitre/data_t/BCM.xlsx'

In [31]:
df = pd.read_excel(file_path)

In [32]:
df.shape[0]

23

In [ ]:
def crawl_news_vietnamnet(ticker='VIC', start_date='01/01/2024 00:00', end_date='01/05/2025 23:59', max_pages=3, delay=2):
    import os
    import pandas as pd
    from datetime import datetime
    import time
    from selenium.webdriver.common.by import By

    file_path = f'D:/thacsi/TAILIEULUANVAN/code/PredictStock_TA_FA_SA/SA/v2/data/news_vnexpress_vietnamnet_tuoitre/data_t/{ticker}.xlsx'

    # Đọc dữ liệu cũ nếu có
    if os.path.exists(file_path):
        df_existing = pd.read_excel(file_path)
        print(f'Len df_existing: {df_existing.shape[0]}')
        existing_urls = set(df_existing['url'].dropna().tolist())
    else:
        df_existing = pd.DataFrame()
        existing_urls = set()


    start_dt = datetime.strptime(start_date, "%d/%m/%Y %H:%M")
    end_dt = datetime.strptime(end_date, "%d/%m/%Y %H:%M")
    new_results = []

    for page in range(0, max_pages):
        print(f"Đang xử lý trang {page+1}...")
        search_url = f"https://vietnamnet.vn/tim-kiem-p{page}?bydaterang=all&cate=000003&newstype=all&od=2&q={ticker}"
        driver.get(search_url)
        time.sleep(delay)

        articles = driver.find_elements(By.CSS_SELECTOR, "div.horizontalPost")

        if not articles:
            print(f"Không có bài viết nào trên trang {page}.")
            break

        for article in articles:
            try:
                a_tag = article.find_elements(By.CSS_SELECTOR, "div.horizontalPost__avt a")[0]
                title = a_tag.get_attribute("title")
                url = a_tag.get_attribute("href")

                if url in existing_urls:
                    print(f"Đã tồn tại: {url}")
                    continue  # bỏ qua bài đã có

                print(f"Đang xử lý: {title}")
                print(f"URL: {url}")

                # Mở bài viết
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[1])
                driver.get(url)
                time.sleep(delay)

                # Lấy thời gian
                try:
                    time_element = driver.find_element(By.CSS_SELECTOR, "div.bread-crumb-detail__time")
                    time_text = convert_time_format_vietnamnet(time_element.text.strip())
                except:
                    time_text = ""
                print(f"Thời gian đăng: {time_text}")

                # Kiểm tra thời gian
                pub_time_date = datetime.strptime(time_text, "%d/%m/%Y %H:%M")
                if pub_time_date > end_dt:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"Bài viết quá mới: {time_text}")
                    continue
                if pub_time_date < start_dt:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"Bài viết quá cũ: {time_text}")
                    return pd.concat([df_existing, pd.DataFrame(new_results)], ignore_index=True)

                # Lấy nội dung
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.maincontent p")
                    description = driver.find_elements(By.CSS_SELECTOR, "h2.content-detail-sapo")
                    content_parts = []

                    if description:
                        content_parts.extend([desc.text.strip() for desc in description if desc.text.strip()])
                    content_parts.extend([p.text.strip() for p in paragraphs if p.text.strip()])

                    content = "\n".join(content_parts)
                except:
                    content = ""

                # Đóng tab
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                # Thêm bài báo mới
                new_results.append({
                    'ticker': ticker,
                    'time': time_text,
                    'tittle': title,
                    'url': url,
                    'Content': clean_text(content)
                })
                print("Đã thêm vào danh sách mới")

            except Exception as e:
                print(f"Lỗi xử lý bài viết: {e}")
                continue

    # Gộp lại với dữ liệu cũ
    df_all = pd.concat([df_existing, pd.DataFrame(new_results)], ignore_index=True)
    df_all.drop_duplicates(subset='url', inplace=True)

    return df_all


In [34]:
df_full = crawl_news_vietnamnet(ticker='BCM', start_date='01/01/2019 00:00', end_date='31/12/2024 00:00', max_pages=80, delay=1.5)

Len df_existing: 23
Đang xử lý trang 1...
Đang xử lý: VN-Index kết phiên rời mốc 1.100 điểm
URL: https://vietnamnet.vn/cap-nhat-chi-so-chung-khoan-vn-index-hom-nay-9-4-2025-2389229.html
Thời gian đăng: 09/04/2025 10:02
Bài viết quá mới: 09/04/2025 10:02
Đang xử lý: VN-Index tiếp tục giảm, xuống gần ngưỡng 1.100 điểm
URL: https://vietnamnet.vn/vn-index-tiep-tuc-giam-sau-huong-ve-1-100-diem-san-200-ty-usd-nong-ruc-2388885.html
Thời gian đăng: 08/04/2025 11:04
Bài viết quá mới: 08/04/2025 11:04
Đang xử lý: VN-Index tiếp tục giảm sâu, mất mốc 1.200 điểm, sàn 200 tỷ USD hút tiền
URL: https://vietnamnet.vn/vn-index-tiep-tuc-giam-sau-mat-moc-1-200-diem-2387824.html
Thời gian đăng: 04/04/2025 11:00
Bài viết quá mới: 04/04/2025 11:00
Đang xử lý: Chứng khoán có lúc giảm tới gần 80 điểm, thanh khoản tăng vọt
URL: https://vietnamnet.vn/chung-khoan-giam-ky-luc-thanh-khoan-tang-vot-2387446.html
Thời gian đăng: 03/04/2025 11:41
Bài viết quá mới: 03/04/2025 11:41
Đã tồn tại: https://vietnamnet.vn/co-p

In [17]:
def convert_time_format_tuoitre(raw_time):
    """
    Chuyển định dạng '30/12/2024 15:30 GMT+7' => '30/12/2024 15:30'
    Trả về cả chuỗi thời gian (string) và đối tượng datetime để so sánh
    """
    try:
        # Loại bỏ phần "GMT+7"
        clean_time = raw_time.replace("GMT+7", "").strip()
        dt = datetime.strptime(clean_time, "%d/%m/%Y %H:%M")
        return dt.strftime("%d/%m/%Y %H:%M")
    except Exception as e:
        print(f"Lỗi chuyển đổi thời gian Tuổi Trẻ: {e}")
        return raw_time

In [35]:
def crawl_news_tuoitre(ticker='VIC', start_date='01/01/2024 00:00', end_date='01/05/2025 23:59', delay=2):
    start_dt = datetime.strptime(start_date, "%d/%m/%Y %H:%M")
    end_dt = datetime.strptime(end_date, "%d/%m/%Y %H:%M")


    driver.get(f"https://tuoitre.vn/tim-kiem.htm?keywords={ticker}&zoneId=11")
    time.sleep(delay)

    # Giai đoạn 1: Load toàn bộ bài viết
    while True:
        print("Bắt đầu load bài viết ....")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)
        try:
            button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.box-viewmore a"))
            )
            button.click()
            time.sleep(delay)
            print("Đang load bài báo ....")
        except:
            break  # không còn nút xem thêm
            print("Đã load xong bài báo")

    # Giai đoạn 2: Duyệt và xử lý bài viết
    results = []
    articles = driver.find_elements(By.CSS_SELECTOR, "div.box-category-item")
    print(f"Tổng số bài viết load được: {len(articles)}")

    for article in articles:
        try:
            a_tag = article.find_element(By.CSS_SELECTOR, "a")
            title = a_tag.get_attribute("title")
            url = a_tag.get_attribute("href")
            print(f"Đang xử lý bài viết: {title}")
            print(f"URL: {url}")

            # Mở tab mới để lấy thời gian và nội dung bài viết
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(url)
            time.sleep(delay)
            print(f"Truy cập bài viết: {url}")

            # Lấy thời gian chính xác
            try:
                time_element = driver.find_element(By.CSS_SELECTOR, 'div[data-role="publishdate"]')
                pub_time = convert_time_format_tuoitre(time_element.text.strip())
                print(f"Thời gian đăng: {pub_time}")
            except Exception as e:
                print(f"Lỗi lấy thời gian: {e}")
                pub_time = None

            if not pub_time:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                continue

            # Kiểm tra điều kiện thời gian
            pub_time_date = datetime.strptime(pub_time, "%d/%m/%Y %H:%M")
            if pub_time_date > end_dt:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(f"Bài viết quá mới không nằm trong khoản thời gian cần crawl: {pub_time}")
                continue
            if pub_time_date < start_dt:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(f"Bài viết quá cũ không nằm trong khoản thời gian cần crawl: {pub_time}")
                break  # các bài sau chắc chắn còn cũ hơn

            # Lấy nội dung bài viết
            try:
                content_parts = []
                desc = driver.find_elements(By.CSS_SELECTOR, "h2.detail-sapo")
                paras = driver.find_elements(By.CSS_SELECTOR, "div.detail-content > p")
                content_parts.extend([d.text.strip() for d in desc if d.text.strip()])
                content_parts.extend([p.text.strip() for p in paras if p.text.strip()])
                content = "\n".join(content_parts)
            except:
                content = ""
            print(f"Nội dung bài viết: {content}")

            # Đóng tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])


            results.append({
                'ticker': ticker,
                'time': pub_time,
                'tittle': title,
                'url': url,
                'Content': clean_text(content)
            })

        except Exception as e:
            print(f"Lỗi khi xử lý bài viết: {e}")
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            continue

    # driver.quit()
    print(f"Tổng số bài viết được lấy: {len(results)}")
    return pd.DataFrame(results)


In [ ]:
def crawl_news_vnexpress_vietnamnet_tuoitre_for_stock(ticker='VIC', start_date='01/01/2024 00:00', end_date='01/05/2025 23:59', delay=2):
    df_vn = crawl_news_vnexpress(ticker, start_date, end_date, max_pages= 38, delay=delay)
    df_vietnamnet = crawl_news_vietnamnet(ticker, start_date, end_date, max_pages= 80, delay=delay)
    df_tuoitre = crawl_news_tuoitre(ticker, start_date, end_date, delay=delay)
    df = pd.concat([df_vn, df_vietnamnet, df_tuoitre], ignore_index=True)
    return df

In [41]:
df = crawl_news_vnexpress_vietnamnet_tuoitre_for_stock("TCB", start_date='01/01/2024 00:00', end_date='01/05/2025 23:59', delay=2)

Len df_existing: 240
Đang xử lý trang 1...
Đang xử lý: 'Đại gia' ngân hàng sẽ bán bảo hiểm nhân thọ vào cuối năm nay
URL: https://vietnamnet.vn/dai-gia-ngan-hang-se-ban-bao-hiem-nhan-tho-vao-cuoi-nam-nay-2395356.html
Thời gian đăng: 26/04/2025 13:49
Đã thêm vào danh sách mới
Đang xử lý: VN-Index tiếp tục giảm, xuống gần ngưỡng 1.100 điểm
URL: https://vietnamnet.vn/vn-index-tiep-tuc-giam-sau-huong-ve-1-100-diem-san-200-ty-usd-nong-ruc-2388885.html
Thời gian đăng: 08/04/2025 11:04
Đã thêm vào danh sách mới
Đang xử lý: Lương duyên gia đình tỷ phú USD Hồ Hùng Anh với 'ông lớn' bất động sản
URL: https://vietnamnet.vn/luong-duyen-cua-gia-dinh-ty-phu-usd-ho-hung-anh-sau-ong-lon-bat-dong-san-2381304.html
Thời gian đăng: 17/03/2025 10:55
Đã thêm vào danh sách mới
Đang xử lý: Masterise Group thành tổ chức liên quan đến Techcombank
URL: https://vietnamnet.vn/ong-lon-bat-dong-san-masterise-group-thanh-to-chuc-lien-quan-den-techcombank-2380951.html
Thời gian đăng: 15/03/2025 09:52
Đã thêm vào danh 

In [42]:
df

,ticker,time,tittle,url,Content
0,TCB,14/10/2020 16:37,Thanh khoản cổ phiếu TCB đạt kỷ lục từ khi lên...,https://vnexpress.net/thanh-khoan-co-phieu-tcb...,"Techcombank vừa có phiên tăng trần, thanh khoả..."
1,TCB,29/09/2020 15:00,CEO Techcombank muốn gom gần 440.000 cổ phiếu TCB,https://vnexpress.net/ceo-techcombank-muon-gom...,Tổng giám đốc Techcombank đăng ký mua 439.000 ...
2,TCB,30/12/2024 05:30,Các tỷ phú giàu nhất Việt Nam 2024: Nhiều tỷ p...,https://vietnamnet.vn/nguoi-giau-nhat-viet-nam...,"Việt Nam có 6 tỷ phú USD vào cuối năm 2024, tổ..."
3,TCB,25/12/2024 16:23,Dòng tiền bất ngờ đổ vào nhóm cổ phiếu đại gia...,https://vietnamnet.vn/dong-tien-bat-ngo-do-vao...,Thị trường chứng khoán ghi nhận dòng tiền tăng...
4,TCB,17/12/2024 05:25,Quá tải người đi cập nhật tài khoản trước hạn ...,https://vietnamnet.vn/qua-tai-nguoi-di-cap-nha...,"Khách hàng lấy số từ đầu giờ sáng, biết còn rấ..."
...,...,...,...,...,...
258,TCB,04/03/2025 20:24,"Cha tỉ phú, mẹ top 10 người giàu nhất sàn, tài...",https://tuoitre.vn/cha-ti-phu-me-top-10-nguoi-...,"Cổ phiếu TCB của Techcombank tăng 5,4% phiên n..."
259,TCB,27/08/2024 21:25,Lộ diện những ‘đại gia’ bất động sản tầm cỡ nắ...,https://tuoitre.vn/lo-dien-nhung-dai-gia-bat-d...,Nhiều ngân hàng đã công bố danh sách cổ đông n...
260,TCB,08/06/2024 12:14,TCB hoàn thành tăng vốn lên 70.450 tỉ đồng,https://tuoitre.vn/tcb-hoan-thanh-tang-von-len...,TCB chính thức tăng vốn điều lệ từ 35.225 tỉ đ...
261,TCB,27/03/2024 21:29,Cổ đông Techcombank được trả cổ tức cả tiền mặ...,https://tuoitre.vn/co-dong-techcombank-duoc-tr...,Techcombank đã gây bất ngờ khi công bố sẽ trìn...


In [51]:
def crawl_news_vnexpress_vietnamnet_tuoitre_for_vn30(tickers, start_date, end_date, roof_path):
  for ticker in tickers:
    try:
      
      # driver = init_driver()
      print(f"Bắt đầu xử lý cho cổ phiếu {ticker}")
      df = crawl_news_vnexpress_vietnamnet_tuoitre_for_stock(ticker, start_date, end_date, delay=1.5)
      os.makedirs(roof_path, exist_ok=True)
      file_path = os.path.join(roof_path, f"{ticker}.xlsx")
      print(f'file path {file_path}')
      with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        df.to_excel(writer, sheet_name='Sheet1', index=False)
      print("Đã lưu file excel cho cổ phiếu " + str(ticker))
      print(f"Số bài báo đã lưu {df.shape[0]}")
      # driver.quit()
      # time.sleep(1)
    except Exception as e:
      print(f"Lỗi khi xử lý cổ phiếu {ticker}: {e}")
      continue

In [ ]:
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']

crawl_news_vnexpress_vietnamnet_tuoitre_for_vn30(tickers, start_date='01/01/2019 00:00', end_date='31/12/2024 00:00', roof_path = r'D:\thacsi\TAILIEULUANVAN\FundamentalAnalysisAndSentimentAnalysis\stock_trading_strategy\PredictStock_TA_FA_SA\SA\v2\data\news_vnexpress_vietnamnet_tuoitre\result')

Bắt đầu xử lý cho cổ phiếu BCM
Len df_existing: 23
Đang xử lý trang 1...
Đang xử lý: VN-Index kết phiên rời mốc 1.100 điểm
URL: https://vietnamnet.vn/cap-nhat-chi-so-chung-khoan-vn-index-hom-nay-9-4-2025-2389229.html
Thời gian đăng: 09/04/2025 10:02
Bài viết quá mới: 09/04/2025 10:02
Đang xử lý: VN-Index tiếp tục giảm, xuống gần ngưỡng 1.100 điểm
URL: https://vietnamnet.vn/vn-index-tiep-tuc-giam-sau-huong-ve-1-100-diem-san-200-ty-usd-nong-ruc-2388885.html
Thời gian đăng: 08/04/2025 11:04
Bài viết quá mới: 08/04/2025 11:04
Đang xử lý: VN-Index tiếp tục giảm sâu, mất mốc 1.200 điểm, sàn 200 tỷ USD hút tiền
URL: https://vietnamnet.vn/vn-index-tiep-tuc-giam-sau-mat-moc-1-200-diem-2387824.html
Thời gian đăng: 04/04/2025 11:00
Bài viết quá mới: 04/04/2025 11:00
Đang xử lý: Chứng khoán có lúc giảm tới gần 80 điểm, thanh khoản tăng vọt
URL: https://vietnamnet.vn/chung-khoan-giam-ky-luc-thanh-khoan-tang-vot-2387446.html
Thời gian đăng: 03/04/2025 11:41
Bài viết quá mới: 03/04/2025 11:41
Đã tồn 